## Modules

In [ ]:
import numpy as np
from GridPythonModule import *
from GPM_wrapper import *

#from tqdm.notebook import tqdm
# ----- Debugger -----
# from IPython.core.debugger import Pdb; Pdb().set_trace()

## Scratch

In [ ]:
mg = Grid(*[[3, 2, 5, 4, 7, 6, 0, 1], [0, 4, 3, 6, 5, 1, 2, 7]]); mg.draw()
print(reduce(mg.get_PD()))

In [ ]:
nontrivknot = None
while nontrivknot is None:
    G = generate_random_grid(20)
    sG = set(sum(simplify_grid(G), []))
    if number_of_components(G) == 1 and sG != {0,1}:
        nontrivknot = G
#---
mg = Grid(*G); mg.draw()
PD = mg.get_PD()
print(reduce(PD))
#---
mgs = Grid(*simplify_grid(G)); mgs.draw()
PDs = mgs.get_PD()
print(reduce(PDs))